In [1]:
#default_exp data.load

# Data load
> Core modifications for making the dataloader work

## TODOs:
* Should not modify main dataloader, but instead subclass it.

In [2]:
#export
from fastai2.vision.all import *

In [3]:
#export
def _zip_collate(t): return Tuple(zip(*t))
def _zip_convert(t): raise NotImplementedError

In [ ]:
#export
def detect_batch_to_samples(b, max_n=10): return L(b).zip()[:max_n]

In [31]:
def batch_to_samples(b, max_n=10):
    "'Transposes' a batch to (at most `max_n`) samples"
    if isinstance(b, Tensor): return retain_types(list(b[:max_n]), [b])
    else:
        res = L(b).map(partial(batch_to_samples,max_n=max_n))
        return retain_types(res.zip(), [b])

In [5]:
#export
class DetectDataLoader(TfmdDL):
    def create_batch(self, b): return (_zip_collate,_zip_convert)[self.prebatched](b)
    
    def _decode_batch(self, b, max_n=9, full=True):
        f = self.after_item.decode
        f = compose(f, partial(getattr(self.dataset,'decode',noop), full = full))
        return L(detect_batch_to_samples(b, max_n=max_n)).map(f)
    
    def show_batch(self, b=None, max_n=9, ctxs=None, show=True, unique=False, **kwargs):
        if unique:
            old_get_idxs = self.get_idxs
            self.get_idxs = lambda: Inf.zeros
        if b is None: b = self.one_batch()
        if not show: return self._pre_show_batch(b, max_n=max_n)
        show = show_batch[type(b[0][0]), type(b[1][0])]
        pb = self._pre_show_batch(b, max_n=max_n)
        show(*pb, ctxs=ctxs, max_n=max_n, **kwargs)
        if unique: self.get_idxs = old_get_idxs

## Export -

In [34]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 03_data.load.ipynb.
Converted 04_data.core.ipynb.
Converted index.ipynb.
